In [99]:
import os
os.chdir('c:\\Users\\utilisateur\\Documents\\GitHub\\ML_Project1\\ML_project1')
         
import sys
sys.path.append(os.getcwd())

In [100]:
import numpy as np
from helpers import *

In [101]:
"""
import pandas as pd

# Load your large dataset
file_path = 'dataset\\x_train.csv'
df = pd.read_csv(file_path)

# Define the number of rows you want in each smaller file
rows_per_file = 50000  # You can change this based on the file size

# Split the dataset
for i in range(0, len(df), rows_per_file):
    # Create a smaller dataset slice
    df_subset = df[i:i+rows_per_file]
    
    # Save this slice to a new CSV file
    output_file = f'small_dataset_part_{i//rows_per_file + 1}.csv'
    df_subset.to_csv(output_file, index=False)
    print(f'Created {output_file}')
"""

"\nimport pandas as pd\n\n# Load your large dataset\nfile_path = 'dataset\\x_train.csv'\ndf = pd.read_csv(file_path)\n\n# Define the number of rows you want in each smaller file\nrows_per_file = 50000  # You can change this based on the file size\n\n# Split the dataset\nfor i in range(0, len(df), rows_per_file):\n    # Create a smaller dataset slice\n    df_subset = df[i:i+rows_per_file]\n    \n    # Save this slice to a new CSV file\n    output_file = f'small_dataset_part_{i//rows_per_file + 1}.csv'\n    df_subset.to_csv(output_file, index=False)\n    print(f'Created {output_file}')\n"

## Load the data 

In [102]:
# Load dataset
file_path = 'dataset\\'
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(file_path)

removal_log = []


In [103]:
removed_missing_values = []
removed_variance = []
removed_correlation = []

In [104]:
def log_and_print_removals(headers, removed_indices, step_name, reason):
    """
    Logs and prints the features removed at each step.

    Args:
        headers (list): The list of feature names.
        removed_indices (list): Indices of features that were removed at the current step.
        step_name (str): The name of the step where the features were removed.
        reason (str): The reason why the features were removed.
    """
    removed_features = [headers[i] for i in removed_indices]
    for feature in removed_features:
        removal_log.append(f"{feature} removed at {step_name}: {reason}")

    print(f"\nStep {step_name} - Reason: {reason}:")
    print(f"Removed {len(removed_features)} features: {removed_features}")

In [105]:
# Function to print kept/removed features after each step
def print_feature_info(headers, kept_indices, step_name):
    """
    Prints the number of features that were kept and removed after each step, 
    along with the respective feature names.

    Args:
        headers (list): The list of feature names.
        kept_indices (list): Indices of features that were kept after the step.
        step_name (str): The name of the step for which this information is printed.
    """
    kept_features = [headers[i] for i in kept_indices]
    removed_features = [headers[i] for i in range(len(headers)) if i not in kept_indices]
    
    print(f"\nStep {step_name}:")
    print(f"Kept {len(kept_features)} features: {kept_features}")
    print(f"Removed {len(removed_features)} features: {removed_features}")

In [106]:
def extract_headers(csv_file_path):
    """
    This function extracts the headers (feature names) from the given CSV file.
    
    Args:
        csv_file_path (str): Path to the CSV file from which to extract headers.
        
    Returns:
        list: A list of headers (feature names), excluding the 'Id' column.
    """
    with open(csv_file_path, 'r') as f:
        reader = csv.reader(f)
        headers = next(reader)  # Read the first row as headers
    return headers[1:]  # Exclude the first column header as it is 'Id'

In [107]:
# Extract headers from x_train.csv
x_train_headers = extract_headers("dataset\\x_train.csv")

In [108]:
# Initialize counters and lists
binary_count = 0
continuous_count = 0
categorical_count = 0

binary_features = []
continuous_features = []
categorical_features = []

# Threshold for distinguishing categorical from continuous
categorical_threshold = 10

# Analyze each feature column-wise
for col_idx in range(x_train.shape[1]):
    column_data = x_train[:, col_idx]
    
    # Remove missing or empty values to avoid counting them
    column_data = column_data[~np.isnan(column_data)]  # Remove NaN values

    # Get unique values
    unique_values = np.unique(column_data)
    unique_count = len(unique_values)

    # Binary feature: 2 unique values
    if unique_count == 2:
        binary_count += 1
        binary_features.append(x_train_headers[col_idx])
    
    # Continuous feature: more than categorical_threshold unique values
    elif unique_count > categorical_threshold:
        continuous_count += 1
        continuous_features.append(x_train_headers[col_idx])
    
    # Categorical feature: 3-10 unique values (or another condition you define)
    else:
        categorical_count += 1
        categorical_features.append(x_train_headers[col_idx])

# Output the counts and the feature types
print(f"Binary features: {binary_count}")
print(f"Continuous features: {continuous_count}")
print(f"Categorical features: {categorical_count}")

# Optionally, display the lists of features by type
print("Binary features:", binary_features)
print("Continuous features:", continuous_features)
print("Categorical features:", categorical_features)

Binary features: 14
Continuous features: 102
Categorical features: 205
Binary features: ['IYEAR', 'DISPCODE', 'PVTRESD1', 'CELLFON3', 'LADULT', 'CADULT', 'PVTRESD2', 'CSTATE', 'SEX', '_DRDXAR1', '_FRTRESP', '_VEGRESP', '_FRT16', '_VEG23']
Continuous features: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'SEQNO', '_PSU', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'HHADULT', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'DIABAGE2', 'CHILDREN', 'WEIGHT2', 'HEIGHT3', 'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUITJU1', 'FRUIT1', 'FVBEANS', 'FVGREEN', 'FVORANG', 'VEGETAB1', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'STRENGTH', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'CRGVREL1', 'CRGVPRB1', 'LONGWTCH', 'ASTHMAGE', 'ASDRVIST', 'ASRCHKUP', 'ASACTLIM', 'SCNTWRK1', 'SCNTLWK1', 'ADPLEASR', 'ADDOWN', 'ADSLEEP', 'ADENERGY', 'ADEAT1', 'ADFAIL', 'ADTHINK', 'ADMOVE', '_STSTR', '_STRWT', '_RAWRAKE', '_WT2

## Step 1 : Remove missing data

In [109]:
def remove_missing_values(headers, data, threshold=0.25):
    """
    Removes features (columns) from the dataset that have more than the specified
    percentage of missing values.

    Args:
        headers (list): The list of feature names.
        data (np.array): The dataset (numpy array) with features as columns.
        threshold (float): The maximum allowed percentage of missing values (default is 25%).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    # Find the percentage of missing values for each column
    missing_values = np.isnan(data).mean(axis=0)
    valid_columns = np.where(missing_values <= threshold)[0]
    removed_columns = np.where(missing_values > threshold)[0]

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_missing_values.extend(removed_features_names)
    
    # Log removed features
    log_and_print_removals(headers, removed_columns, "1: Remove Missing Values", "Too many missing values")
    
    # Filter out the columns with too many missing values
    filtered_data = data[:, valid_columns]
    return valid_columns, filtered_data

# Apply Step 1: Missing Values
valid_columns, x_train = remove_missing_values(x_train_headers, x_train)
x_train_headers = [x_train_headers[i] for i in valid_columns]  # Update headers after removing columns



Step 1: Remove Missing Values - Reason: Too many missing values:
Removed 178 features: ['CTELENUM', 'PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT', 'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'POORHLTH', 'BPMEDS', 'ASTHNOW', 'DIABAGE2', 'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'PREGNANT', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'LMTJOIN3', 'ARTHDIS2', 'ARTHSOCL', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'PDIABTST', 'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'EYEEXAM', 'DIABEYE', 'DIABEDU', 'CAREGIV1', 'CRGVREL1', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB1', 'CRGVPERS', 'CRGVHOUS', 'CRGVMST2', 'CRGVEXPT', 'VIDFCLT2', 'VIREDIF3', 'VIPRFVS2', 'VINOCRE2', 'VIEYEXM2', 'VIINSUR2', 'VICTRCT4', 'VIGLUMA2', 'VIMACDG2', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHEL

## Step 2 : Variance Thresholding

In [110]:
def variance_thresholding(headers, data, threshold=0.01):
    """
    Removes features that have variance below a specified threshold.

    Args:
        headers (list): The list of feature names.
        data (np.array): The dataset (numpy array) with features as columns.
        threshold (float): The minimum variance threshold (default is 1%).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    # Compute variance for each column
    variances = np.nanvar(data, axis=0)
    valid_columns = np.where(variances >= threshold)[0]
    removed_columns = np.where(variances < threshold)[0]

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_variance.extend(removed_features_names)
    
    # Log removed features
    log_and_print_removals(headers, removed_columns, "2: Variance Thresholding", "Low variance")
    
    # Keep only columns with variance above the threshold
    filtered_data = data[:, valid_columns]
    return valid_columns, filtered_data

# Apply Step 2: Variance Thresholding
valid_columns, x_train = variance_thresholding(x_train_headers, x_train)
x_train_headers = [x_train_headers[i] for i in valid_columns]  # Update headers after removing columns




Step 2: Variance Thresholding - Reason: Low variance:
Removed 2 features: ['_FRT16', '_VEG23']


## Step 3 : Correlation analysis

In [111]:
def correlation_analysis(headers, x_data, y_data, low_threshold=0.05, high_threshold=0.9):
    """
    Removes features that have low correlation with the outcome variable (y_data),
    or that are highly correlated with other features.

    Args:
        headers (list): The list of feature names.
        x_data (np.array): The feature dataset (numpy array) with features as columns.
        y_data (np.array): The outcome variable (labels) to correlate with.
        low_threshold (float): Minimum Pearson correlation threshold (default is 0.05).
        high_threshold (float): Maximum Pearson correlation threshold (default is 0.9).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    valid_columns = []
    removed_columns = []

    # Ensure feature and outcome have the same non-NaN indices
    for i in range(x_data.shape[1]):
        feature = x_data[:, i]

        # Create a mask where both feature and y_data are non-NaN
        non_nan_mask = ~np.isnan(feature) & ~np.isnan(y_data)

        # Apply mask to both feature and y_data
        feature_clean = feature[non_nan_mask]
        y_data_clean = y_data[non_nan_mask]

        # Compute correlation if both feature_clean and y_data_clean have enough data
        if len(feature_clean) > 1 and len(y_data_clean) > 1:
            correlation = np.corrcoef(feature_clean, y_data_clean)[0, 1]

            # Check if the correlation is within the desired range
            if abs(correlation) >= low_threshold and abs(correlation) <= high_threshold:
                valid_columns.append(i)
            else:
                removed_columns.append(i)
        else:
            removed_columns.append(i)

    # Log removed features
    log_and_print_removals(headers, removed_columns, "3: Correlation Analysis", "Low or high correlation")

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_correlation.extend(removed_features_names)

    # Filter valid columns
    filtered_data = x_data[:, valid_columns]
    return valid_columns, filtered_data

# Apply Step 3: Correlation Analysis
valid_columns, x_train = correlation_analysis(x_train_headers, x_train, y_train)
x_train_headers = [x_train_headers[i] for i in valid_columns]  # Update headers after removing columns




Step 3: Correlation Analysis - Reason: Low or high correlation:
Removed 90 features: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE', 'MENTHLTH', 'HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'ASTHMA3', 'ADDEPEV2', 'MARITAL', 'RENTHOM1', 'INCOME2', 'WEIGHT2', 'HEIGHT3', 'BLIND', 'DECIDE', 'USENOW3', 'FRUITJU1', 'FRUIT1', 'FVBEANS', 'FVGREEN', 'FVORANG', 'VEGETAB1', 'EXERANY2', 'SEATBELT', 'HIVTST6', 'QSTLANG', '_STSTR', '_STRWT', '_RAWRAKE', '_WT2RAKE', '_DUALUSE', '_LLCPWT', '_LTASTH1', '_CASTHM1', '_ASTHMS1', '_PRACE1', '_MRACE1', '_HISPANC', '_RACE', '_RACEG21', '_RACEGR3', '_RACE_G1', 'HTIN4', 'HTM4', '_RFBMI5', '_SMOKER3', '_RFSMOK3', 'DRNKANY5', 'DROCDY3_', '_RFBING5', '_DRNKWEK', '_RFDRHV5', 'FTJUDA1_', 'FRUTDA1_', 'BEANDAY_', 'GRENDAY_', 'ORNGDAY_', 'VEGEDA1_', '_MISFRTN', '_MISVEGN', '_FRTRESP', '_VEGRESP', '_FRUTSUM', '_VEGESUM', '_FRTLT1', '_VEGLT1', '_FRUITEX', '_VEGETEX', '_TOTINDA', 'MAXVO2_', 'FC60_', 'STRFREQ_', 'PAMISS1_', '_PACAT1', '_PAINDX1', '_PA150R2', '

In [112]:
# Print the final removal log for interpretability
print("\nRemoval Log:")
for entry in removal_log:
    print(entry)

print("Preprocessing complete. Cleaned dataset saved as 'cleaned_dataset.csv'.")


Removal Log:
CTELENUM removed at 1: Remove Missing Values: Too many missing values
PVTRESD1 removed at 1: Remove Missing Values: Too many missing values
COLGHOUS removed at 1: Remove Missing Values: Too many missing values
STATERES removed at 1: Remove Missing Values: Too many missing values
CELLFON3 removed at 1: Remove Missing Values: Too many missing values
LADULT removed at 1: Remove Missing Values: Too many missing values
NUMADULT removed at 1: Remove Missing Values: Too many missing values
NUMMEN removed at 1: Remove Missing Values: Too many missing values
NUMWOMEN removed at 1: Remove Missing Values: Too many missing values
CTELNUM1 removed at 1: Remove Missing Values: Too many missing values
CELLFON2 removed at 1: Remove Missing Values: Too many missing values
CADULT removed at 1: Remove Missing Values: Too many missing values
PVTRESD2 removed at 1: Remove Missing Values: Too many missing values
CCLGHOUS removed at 1: Remove Missing Values: Too many missing values
CSTATE remov

In [113]:
print(x_train_headers)

['SEQNO', '_PSU', 'GENHLTH', 'PHYSHLTH', 'CHECKUP1', 'BPHIGH4', 'BLOODCHO', 'CHOLCHK', 'TOLDHI2', 'CVDSTRK3', 'CHCSCNCR', 'CHCOCNCR', 'CHCCOPD1', 'HAVARTH3', 'CHCKIDNY', 'DIABETE3', 'SEX', 'EDUCA', 'VETERAN3', 'EMPLOY1', 'CHILDREN', 'INTERNET', 'QLACTLM2', 'USEEQUIP', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'SMOKE100', 'ALCDAY5', 'STRENGTH', 'FLUSHOT6', 'PNEUVAC3', 'QSTVER', '_RFHLTH', '_HCVU651', '_RFHYPE5', '_CHOLCHK', '_RFCHOL', '_DRDXAR1', '_AGEG5YR', '_AGE65YR', '_AGE80', '_AGE_G', 'WTKG3', '_BMI5', '_BMI5CAT', '_CHLDCNT', '_EDUCAG', '_INCOMG', '_LMTACT1', '_LMTSCL1']


In [114]:
removed_features = removed_missing_values + removed_variance + removed_correlation
print(removed_features)


['CTELENUM', 'PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT', 'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'POORHLTH', 'BPMEDS', 'ASTHNOW', 'DIABAGE2', 'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'PREGNANT', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'LMTJOIN3', 'ARTHDIS2', 'ARTHSOCL', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'PDIABTST', 'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'EYEEXAM', 'DIABEYE', 'DIABEDU', 'CAREGIV1', 'CRGVREL1', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB1', 'CRGVPERS', 'CRGVHOUS', 'CRGVMST2', 'CRGVEXPT', 'VIDFCLT2', 'VIREDIF3', 'VIPRFVS2', 'VINOCRE2', 'VIEYEXM2', 'VIINSUR2', 'VICTRCT4', 'VIGLUMA2', 'VIMACDG2', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'CDDISCUS', 'WTCHSALT', 'LONGWTCH', 'DRADVISE', 'ASTHMAGE', 'ASATTACK', 

## For us

In [115]:
import pandas as pd

# Load your Excel file with all features and their meanings
input_file_path = 'BRFSS_2015_Cleaned_Parameters_excel.xlsx'
df_parameters = pd.read_excel(input_file_path)

# List of conserved features to filter
conserved_features = x_train_headers

# Filter the DataFrame for conserved features
df_conserved = df_parameters[df_parameters['Parameter'].isin(conserved_features)]

# Save the filtered data to a new Excel file
output_file_path = 'conserved_features_output.xlsx'
df_conserved.to_excel(output_file_path, index=False)

print(f"Excel file generated and saved as {output_file_path}")

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\3861114181.py:15: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_conserved.to_excel(output_file_path, index=False)


Excel file generated and saved as conserved_features_output.xlsx


In [119]:
# Parse the removal log into a structured format
removed_features_ = []
removal_reasons = []

for log in removal_log:
    feature, reason = log.split(" removed at ")[0], log.split(": ", 1)[1]
    removed_features_.append(feature)
    removal_reasons.append(reason)

# Filter the DataFrame to include only the removed features
df_removed_info = df_parameters[df_parameters['Parameter'].isin(removed_features_)]

# Add the reason for removal to the DataFrame
df_removed_info['Reason of Removal'] = df_removed_info['Parameter'].map(dict(zip(removed_features_, removal_reasons)))

# Save the updated DataFrame with feature names, descriptions, value meanings, and reasons for removal to an Excel file
output_file_path = 'removed_features_with_description_log.xlsx'
df_removed_info.to_excel(output_file_path, index=False)

print(f"Excel file generated and saved as {output_file_path}")

Excel file generated and saved as removed_features_with_description_log.xlsx


C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\2870947207.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_removed_info['Reason of Removal'] = df_removed_info['Parameter'].map(dict(zip(removed_features_, removal_reasons)))
C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\2870947207.py:18: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_removed_info.to_excel(output_file_path, index=False)
